In [2]:
from local.node import Node
from local.constnode import ConstNode
from local.varnode import VarNode
from local.funcnode import FuncNode
from local.func import *
from local.treeoptimizer import TreeOptimizer
from local.crossover import *
from local.Optimizer import *

import os
import copy
import random
import pickle
import graphviz
import datetime
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, metrics, optimizers

In [30]:

BATCH_SIZE = 512
EPOCHS = 10
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train, x_test = x_train[..., tf.newaxis] / 255.0, x_test[..., tf.newaxis] / 255.0

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

    return train_dataset, test_dataset

def build_model():
    input = layers.Input(shape = (32, 32, 3))
    x = layers.Conv2D(32, 3, 2)(input)
    x = layers.LeakyReLU()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units = 32, activation = "relu")(x)
    output = layers.Dense(units = 10, activation = "softmax")(x)

    return models.Model(input, output)

train_dataset, test_dataset = load_data()

model = build_model()
model.save_weights("model.keras")

In [31]:
# tensorflow
optimizer = optimizers.SGD(learning_rate = 0.001, momentum=0.9)

model.load_weights("model.keras")

loss_obj = metrics.Mean()
accuracy_obj = metrics.SparseCategoricalAccuracy()
val_loss_obj = metrics.SparseCategoricalCrossentropy()
val_accuracy_obj = metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(X, Y):
    with tf.GradientTape() as tape:
        pred = model(X)
        loss = losses.SparseCategoricalCrossentropy()(Y, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    loss_obj(loss)
    accuracy_obj(Y, pred)

@tf.function
def test_step(X, Y):
    pred = model(X)
    
    val_loss_obj(Y, pred)
    val_accuracy_obj(Y, pred)

for i in range(EPOCHS):
    
    loss_obj.reset_state()
    accuracy_obj.reset_state()
    val_loss_obj.reset_state()
    val_accuracy_obj.reset_state()

    for X, Y in tqdm(train_dataset):
        train_step(X, Y)
    
    for X, Y in tqdm(test_dataset):
        test_step(X, Y)

    print(f"epoch : {i}, loss : {float(loss_obj.result())}, accuracy : {float(accuracy_obj.result())}")
    print(f"val_loss : {float(val_loss_obj.result())}, val_accuracy : {float(val_accuracy_obj.result())}")

100%|██████████| 20/20 [00:00<00:00, 53.69it/s]


epoch : 0, loss : 2.2189130783081055, accuracy : 0.17844000458717346
val_loss : 2.1440818309783936, val_accuracy : 0.2289000004529953


100%|██████████| 20/20 [00:00<00:00, 119.00it/s]


epoch : 1, loss : 2.0837299823760986, accuracy : 0.26287999749183655
val_loss : 2.032944917678833, val_accuracy : 0.2831000089645386


100%|██████████| 20/20 [00:00<00:00, 113.30it/s]


...

epoch : 7, loss : 1.8327218294143677, accuracy : 0.36395999789237976
val_loss : 1.827788233757019, val_accuracy : 0.36059999465942383


100%|██████████| 20/20 [00:00<00:00, 120.27it/s]


epoch : 8, loss : 1.8154494762420654, accuracy : 0.3697200119495392
val_loss : 1.811552882194519, val_accuracy : 0.3659999966621399


100%|██████████| 20/20 [00:00<00:00, 114.25it/s]

epoch : 9, loss : 1.7997026443481445, accuracy : 0.37553998827934265
val_loss : 1.7954643964767456, val_accuracy : 0.37220001220703125


In [12]:
optimizer = RMSProp(learning_rate = 0.01)

model.load_weights("model.keras")

loss_obj = metrics.Mean()
accuracy_obj = metrics.SparseCategoricalAccuracy()
val_loss_obj = metrics.SparseCategoricalCrossentropy()
val_accuracy_obj = metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(X, Y):
    with tf.GradientTape() as tape:
        pred = model(X)
        loss = losses.SparseCategoricalCrossentropy()(Y, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    loss_obj(loss)
    accuracy_obj(Y, pred)

@tf.function
def test_step(X, Y):
    pred = model(X)
    
    val_loss_obj(Y, pred)
    val_accuracy_obj(Y, pred)

for i in range(EPOCHS):
    
    loss_obj.reset_state()
    accuracy_obj.reset_state()
    val_loss_obj.reset_state()
    val_accuracy_obj.reset_state()

    for X, Y in tqdm(train_dataset):
        train_step(X, Y)
    
    for X, Y in tqdm(test_dataset):
        test_step(X, Y)

    print(f"epoch : {i}, loss : {float(loss_obj.result())}, accuracy : {float(accuracy_obj.result())}")
    print(f"val_loss : {float(val_loss_obj.result())}, val_accuracy : {float(val_accuracy_obj.result())}")

100%|██████████| 20/20 [00:00<00:00, 56.15it/s]


epoch : 0, loss : 4.034092426300049, accuracy : 0.10840000212192535
val_loss : 2.302746295928955, val_accuracy : 0.09989999979734421


100%|██████████| 20/20 [00:00<00:00, 119.27it/s]


epoch : 1, loss : 2.4209821224212646, accuracy : 0.09694000333547592
val_loss : 2.3029158115386963, val_accuracy : 0.10000000149011612


100%|██████████| 20/20 [00:00<00:00, 120.63it/s]


...

epoch : 7, loss : 2.303037643432617, accuracy : 0.09718000143766403
val_loss : 2.3028018474578857, val_accuracy : 0.10000000149011612


100%|██████████| 20/20 [00:00<00:00, 119.84it/s]


epoch : 8, loss : 2.303037643432617, accuracy : 0.09718000143766403
val_loss : 2.3028018474578857, val_accuracy : 0.10000000149011612


100%|██████████| 20/20 [00:00<00:00, 117.79it/s]


epoch : 9, loss : 2.303037643432617, accuracy : 0.09718000143766403
val_loss : 2.3028018474578857, val_accuracy : 0.10000000149011612
